In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

#use fastai v2
from fastai.vision.all import *  
from tqdm import tqdm
from glob import glob
import cv2
import  os
import  zipfile

In [ ]:
df = pd.read_csv("../input/uw-madison-gi-tract-image-segmentation/train.csv", low_memory = False)
ss_df = pd.read_csv("../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv", low_memory = False)

In [ ]:
DEBUG=len(ss_df)==0
TRAIN_DIR = "../input/uw-madison-gi-tract-image-segmentation/train"
TEST_DIR = "../input/uw-madison-gi-tract-image-segmentation/test"
all_train_images = glob(os.path.join(TRAIN_DIR, "**", "*.png"), recursive = True)
all_test_images = glob(os.path.join(TEST_DIR, "**", "*.png"), recursive = True)

if DEBUG:
    TEST_DIR = TRAIN_DIR
    all_test_images = all_train_images
    ss_df = df
    ss_df = ss_df[["id", "class"]]
    ss_df["predicted"] = ""

拼接处理df和ss_df，实际上df在infer部分完全没有用到，但是可以辅助来看拼接效果,得出baseline

In [ ]:
df["case_no"] = df["id"].apply(lambda x: x.split("_")[0]) 
df["case_no_int"] = df["case_no"].apply(lambda x: x.replace("case", ""))
df["day_no"] = df["id"].apply(lambda x: x.split("_")[1])
df["day_no_int"] = df["day_no"].apply(lambda x: x.replace("day", ""))
df["slice_id"] = df["id"].apply(lambda x: x.split("_")[2:])
df["slice_id"] = df["slice_id"].apply(lambda x: x[0] + "_" + x[1])
df["_partial_ident"] = (all_train_images[0].rsplit("/", 4)[0] + "/" + df["case_no"] + "/" + df["case_no"] + "_" + df["day_no"] + "/scans/" + df["slice_id"])
_tmp_merge_df = pd.DataFrame({"_partial_ident" : [x.rsplit("_", 4)[0] for x in all_train_images],"f_path" : all_train_images})
df = df.merge(_tmp_merge_df, on = "_partial_ident").drop(columns = ["_partial_ident"])

df["slice_h"] = df["f_path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[1]))
df["slice_w"] = df["f_path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[2]))
df["px_spacing_h"] = df["f_path"].apply(lambda x: float(x[:-4].rsplit("_", 4)[3]))
df["px_spacing_w"] = df["f_path"].apply(lambda x: float(x[:-4].rsplit("_", 4)[4]))
l_bowel_df = df[df["class"] == "large_bowel"][["id", "segmentation"]].rename(columns = {"segmentation" : "large_bowel_seg"})
s_bowel_df = df[df["class"] == "small_bowel"][["id", "segmentation"]].rename(columns = {"segmentation" : "small_bowel_seg"})
stomach_df = df[df["class"] == "stomach"][["id", "segmentation"]].rename(columns = {"segmentation" : "stomach_seg"})
df = df.merge(l_bowel_df, on = "id", how = "left")
df = df.merge(s_bowel_df, on = "id", how = "left")
df = df.merge(stomach_df, on = "id", how = "left")
df = df.drop_duplicates(subset = "id").reset_index(drop = True)
df["large_bowel_seg_flag"] = df["large_bowel_seg"].apply(lambda x: not pd.isna(x))
df["small_bowel_seg_flag"] = df["small_bowel_seg"].apply(lambda x: not pd.isna(x))
df["stomach_seg_flag"] = df["stomach_seg"].apply(lambda x: not pd.isna(x))
df["n_segs"] = df["large_bowel_seg_flag"].astype(int) + df["small_bowel_seg_flag"].astype(int) + df["stomach_seg_flag"].astype(int)
display(df.head())

In [ ]:
#同样的操作发生在ss_df上
ss_df["case_no"] = ss_df["id"].apply(lambda x: x.split("_")[0]) 
ss_df["case_no_int"] = ss_df["case_no"].apply(lambda x: x.replace("case", ""))
ss_df["day_no"] = ss_df["id"].apply(lambda x: x.split("_")[1])
ss_df["day_no_int"] = ss_df["day_no"].apply(lambda x: x.replace("day", ""))
ss_df["slice_id"] = ss_df["id"].apply(lambda x: x.split("_")[2:])
ss_df["slice_id"] = ss_df["slice_id"].apply(lambda x: x[0] + "_" + x[1])
ss_df["_partial_ident"] = (all_test_images[0].rsplit("/", 4)[0] + "/" + ss_df["case_no"] + "/" + ss_df["case_no"] + "_" + ss_df["day_no"] + "/scans/" + ss_df["slice_id"])
_tmp_merge_df_ss = pd.DataFrame({"_partial_ident" : [x.rsplit("_", 4)[0] for x in all_test_images],"f_path" : all_test_images})
ss_df = ss_df.merge(_tmp_merge_df_ss, on = "_partial_ident").drop(columns = ["_partial_ident"])
ss_df["slice_h"] = ss_df["f_path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[1]))
ss_df["slice_w"] = ss_df["f_path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[2]))

ss_df["px_spacing_h"] = ss_df["f_path"].apply(lambda x: float(x[:-4].rsplit("_", 4)[3]))
ss_df["px_spacing_w"] = ss_df["f_path"].apply(lambda x: float(x[:-4].rsplit("_", 4)[4]))
display(ss_df.head())

In [ ]:
#根据ident进行map
ss_df["ident"] = ss_df["case_no"].astype(str)+"-"+ss_df["day_no"].astype(str)+"-"+ss_df["slice_id"].astype(str)+"-"+ss_df["slice_h"].astype(str)+"-"+ss_df["slice_w"].astype(str)+"-"+ss_df["px_spacing_h"].astype(str)+"-"+ss_df["px_spacing_w"].astype(str)+"-"+ss_df["class"].astype(str)

进行模型Infer

In [ ]:
binary = DataBlock(blocks=(ImageBlock, MaskBlock( ['Background', 'stomach', 'large_bowel','small_bowel'])),    
                   get_items=get_image_files,    #x is obtained in get_image_files
                   splitter=RandomSplitter(),    #Random segmentation
                   item_tfms=Resize(160),   #as The image size is inconsistent，'128'will be a problem    
                   batch_tfms=[Normalize.from_stats(*imagenet_stats)])   

In [ ]:
dls = binary.dataloaders(TEST_DIR,bs=3)
#dls.show_batch( vmin=0, vmax=3)

In [ ]:
#p = Path("/root/.cache/torch/hub/checkpoints")
#p.mkdir(parents=True)
#!cp ../input/resnet34/resnet34-b627a593.pth /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth

In [ ]:
!cp -r ../input/pytorch-segmentation-models-lib/ ./
!pip install -q ./pytorch-segmentation-models-lib/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4
!pip install -q ./pytorch-segmentation-models-lib/efficientnet_pytorch-0.6.3/efficientnet_pytorch-0.6.3
!pip install -q ./pytorch-segmentation-models-lib/timm-0.4.12-py3-none-any.whl
!pip install -q ./pytorch-segmentation-models-lib/segmentation_models_pytorch-0.2.0-py3-none-any.whl

In [ ]:
#learn = unet_learner(dls,resnet34,metrics=DiceMulti)
#learn = unet_learner(dls, resnet34, metrics=DiceMulti, self_attention=True, act_cls=Mish, opt_func=ranger)
#learn = load_learner(model_name)
import dill
import segmentation_models_pytorch as smp
learn = load_learner('../input/uwmgit-train-pytorch-fastai/learn_smp_resnet_20epoch.pkl', pickle_module=dill)

In [ ]:
#!mkdir models
#!cp ../input/uwmgit-train-pytorch-fastai/models/model2.pth ./models/model2.pth
#learn.load('model2')

In [ ]:
def rle_encode(mask):
    pixels = mask.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return runs

In [ ]:
result_df = ss_df.copy()
ss_df = ss_df.drop_duplicates(subset = "id").reset_index(drop = True) #去掉重复项
batch = (int)(len(ss_df) /1000)
remainder = len(ss_df) - batch * 1000
print("batch is ",batch,"remainder is ",remainder)

In [ ]:
#生成一个dict用于最后结果输出
slice_px_map = {}

In [ ]:
def make_slice_px_map(test_csv_part):
    test_dl = learn.dls.test_dl(test_csv_part['f_path'])
    preds=[]
    preds = learn.get_preds(dl=test_dl)
    print(ibatch,time.asctime(time.localtime(time.time()))) #显示当前时间
    for idx in range(len(test_csv_part)):
        true_index = len(test_csv_part)*ibatch+idx
        strId = ss_df['id'][true_index]
        case_no = ss_df['case_no'][true_index]
        day_no =  ss_df['day_no'][true_index]
        slice_id = ss_df['slice_id'][true_index]
        slice_h =  ss_df['slice_h'][true_index]
        slice_w = ss_df['slice_w'][true_index]
        px_spacing_h = ss_df['px_spacing_h'][true_index]
        px_spacing_w = ss_df['px_spacing_w'][true_index]
        #large_bowel 2
        submit_np = np.array(preds[0][idx][2]<0.5).astype(np.uint8) 
        msk = PILMask.create(submit_np)
        msk = msk.resize((slice_w,slice_h),Image.ANTIALIAS)  
        submit_np2 = np.array(msk)
        rle = rle_encode(submit_np2)
        rle = ' '.join(str(x) for x in rle)
        if(len(rle)<10): #必须添加
            rle=""
        slice_px_map[f"{case_no}-{day_no}-{slice_id}-{slice_h}-{slice_w}-{px_spacing_h}-{px_spacing_w}-large_bowel"] = rle
        #small_bowel 3
        submit_np = np.array(preds[0][idx][3]<0.5).astype(np.uint8) 
        msk = PILMask.create(submit_np)
        msk = msk.resize((slice_w,slice_h),Image.ANTIALIAS)  
        submit_np2 = np.array(msk)
        rle = rle_encode(submit_np2)
        rle = ' '.join(str(x) for x in rle)
        if(len(rle)<10):
            rle=""
        slice_px_map[f"{case_no}-{day_no}-{slice_id}-{slice_h}-{slice_w}-{px_spacing_h}-{px_spacing_w}-small_bowel"] = rle
        #stomach 1
        submit_np = np.array(preds[0][idx][1]<0.5).astype(np.uint8) 
        msk = PILMask.create(submit_np)
        msk = msk.resize((slice_w,slice_h),Image.ANTIALIAS)  
        submit_np2 = np.array(msk)
        rle = rle_encode(submit_np2)
        rle = ' '.join(str(x) for x in rle)
        if(len(rle)<10):
            rle=""   
        slice_px_map[f"{case_no}-{day_no}-{slice_id}-{slice_h}-{slice_w}-{px_spacing_h}-{px_spacing_w}-stomach"] = rle
    return

In [ ]:
for ibatch in range(batch):
    test_csv_part = ss_df[1000*ibatch:1000*(ibatch+1)]
    make_slice_px_map(test_csv_part)

In [ ]:
#处理最后部分
test_csv_part = ss_df[1000*batch:len(ss_df)]
make_slice_px_map(test_csv_part)

In [ ]:
result_df["predicted"] = result_df["ident"].map(slice_px_map)

In [ ]:
result_df = result_df[["id", "class", "predicted"]]
result_df.to_csv("submission.csv", index = False)
display(result_df.head())